In [1]:
import pandas as pd
from datetime import datetime
import re

rawpath = "../raw"

In [7]:
df = pd.read_excel(f"{rawpath}/riesaufdrucke_neu.xlsx", names=["kasten","signatur","entstehungszeit","entstehungszeit_lang","datentraeger","sprache","land","papiermacher_idn","papiermuehle_idn","papiermacher","papiermuehle","motiv","text","ort","Papiermuehle_4046","umfang","blattmass","bildmass","plattenmass","anmerkungen","komm_1","komm_2","quelle","bezugswerke","schlagwortfolge","wz_motiv","technik","druckfarbe","ort_idn","provenienz_idn","provenienz","komm_extern","8034"], dtype={"papiermacher_idn":"string","papiermuehle_idn":"string", "quelle":"string", "bezugswerke":"string", "schlagwortfolge":"string", "wz_motiv":"string", "technik":"string", "druckfarbe":"string", "ort_idn":"string", "provenienz_idn":"string"})
df = df.apply(lambda x: x.str.strip() if (x.dtype == "object") or (x.dtype == "string") else x)
# df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   kasten                122 non-null    object
 1   signatur              122 non-null    object
 2   entstehungszeit       91 non-null     object
 3   entstehungszeit_lang  99 non-null     object
 4   datentraeger          122 non-null    object
 5   sprache               122 non-null    object
 6   land                  122 non-null    object
 7   papiermacher_idn      62 non-null     string
 8   papiermuehle_idn      80 non-null     string
 9   papiermacher          122 non-null    object
 10  papiermuehle          122 non-null    object
 11  motiv                 122 non-null    object
 12  text                  117 non-null    object
 13  ort                   122 non-null    object
 14  Papiermuehle_4046     122 non-null    object
 15  umfang                122 non-null    ob

In [8]:
def val_idn(idn):
    if len(idn) < 9:
        return (9 - len(idn)) * '0' + idn
    else:
        return idn

In [9]:
def pica_schreiben(row):
    
    pica = """0500 Pa
0501 unbewegtes Bild$bsti
0501 text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
"""
    if year := re.findall(r"\d{4}", str(row.entstehungszeit)):
        if len(year) == 2:
            pica += f"1100 {year[0]}$n[zwischen {year[0]} und {year[1]}]\n1110 {row.entstehungszeit_lang}*{row.entstehungszeit}$a{year[0]}$b{year[1]}$4vzta\n"
        
        elif len(year) == 1:
            pica += f"1100 {year[0]}$n{row.entstehungszeit}\n1110 {row.entstehungszeit_lang}*{row.entstehungszeit}$4vzta\n"
    
    elif year := re.findall(r"\d{2}X{2}", str(row.entstehungszeit)):
        pica += f"1100 {row.entstehungszeit}\n1110 {row.entstehungszeit_lang}*{row.entstehungszeit}$4vzta\n"
    
    pica += f"""1130 {row.datentraeger}
1131 !041454197!
1132 a1-analog;a2-druck;f1-bild;f2-blatt;o-org
1500 /1zxx
1700 {row.land}
"""
    if pd.notna(row.papiermacher_idn):
        counter = 0
        for papiermacher in row.papiermacher_idn.split("\n"):
            pica += f"30{counter}0 !{val_idn(papiermacher)}!$BPapiermacher$4ppm\n"
            counter += 1
    
    if pd.notna(row.papiermuehle_idn):
        pica += f"3100 !{val_idn(row.papiermuehle_idn)}!$BGeistiger Schöpfer$4cre\n"
    
    # Papiermacher bekannt
    if (row.papiermacher != "[Nicht ermittelbar]"):
        pica += f"4000 Riesaufdruck des Papiermachers {row.papiermacher}, Motiv: {row.motiv}"
        
        # Papiermühle bekannt?
        if row.papiermuehle != "[Nicht ermittelbar]":
            pica += f" / {row.papiermuehle}"
        
        pica += "\n"
      
    # Papiermacher unbekannt
    elif (row.papiermacher == "[Nicht ermittelbar]"):
        pica += f"4000 Riesaufdruck eines unbekannten Papiermachers, Motiv: {row.motiv}"
        
        # Papiermühle bekannt?
        if row.papiermuehle != "[Nicht ermittelbar]":
            pica += f" / {row.papiermuehle}"
        
        pica += "\n"

    pica += f"4019 Druckgrafik$Bobja\n"

    if pd.notna(row.text):
        pica += f"4019 {row.text}$Bdevi\n"
    
    pica += f"""4046 {row.ort} : {row.Papiermuehle_4046}
4060 1 ungezähltes Blatt
"""
    blattmasse = [x.replace(" cm", "").replace(",","") for x in row.blattmass.split(" x ")]
    pica += f"4062 {blattmasse[0]} x {blattmasse[1]} mm$b{blattmasse[0]}$h{blattmasse[1]}$4bltt\n"

    bildmasse = [x.replace(" cm", "").replace(",","") for x in row.bildmass.split(" x ")]
    pica += f"4062 {bildmasse[0]} x {bildmasse[1]} mm$b{bildmasse[0]}$h{bildmasse[1]}$4bld\n"

    if pd.notna(row.plattenmass):
        plattenmasse = [x.replace(" cm", "").replace(",","") for x in row.plattenmass.split(" x ")]
        pica += f"4062 {plattenmasse[0]} x {plattenmasse[1]} mm$b{plattenmasse[0]}$h{plattenmasse[1]}$4drck\n"

    pica += "4105 !1257829130!\n"
    
    if pd.notna(row.anmerkungen):
        pica += f"4201 {row.anmerkungen}\n"
    
    if pd.notna(row.komm_1):
        pica += f"4201 {row.komm_1}\n"
    
    if pd.notna(row.komm_2):
        pica += f"4201 {row.komm_2}\n"

    if pd.notna(row.quelle):
        pica += f"4276 !{val_idn(row.quelle)}!$4lita\n4276 !{val_idn(row.quelle)}!$4abba\n"
    
    if pd.notna(row.bezugswerke):
        pica += f"4276 {val_idn(row.bezugswerke)}$4rela\n"
    
    if pd.notna(row.schlagwortfolge):
        counter = 0
        for schlagwort in row.schlagwortfolge.split("\n"):
            pica += f"510{counter} !{val_idn(schlagwort)}!\n"
            if counter < 5:
                counter += 1
    
    if pd.notna(row.wz_motiv):
        for motiv in row.wz_motiv.split("\n"):
            pica += f"5320 !{val_idn(motiv)}!\n"

    pica += "5590 [Objektgattung]\n5590 !042115752!\n"

    pica += f"""5591 [Technik]
5591 !{val_idn(row.technik)}!
"""
    if pd.notna(row.druckfarbe):
        pica += f"5592 [Druck]\n5592 !041507266!\n"

        for farbe in row.druckfarbe.split("\n"):
            pica += f"5592 !{val_idn(farbe)}!\n"

    if pd.notna(row.ort_idn):
        pica += f"5593 [Entstehungsort]\n5593 !{val_idn(row.ort_idn)}!\n"
    
    pica += f"""7001 {now.strftime('%d-%m-%y')} : x
4800 !009033645!
6710 !1032059397!
"""
    if pd.notna(row['provenienz_idn']):
        pica += f"6800 [Provenienz]\n6800 !{val_idn(row['provenienz_idn'])}!\n6800 {row.provenienz}\n"
    
    pica += f"7100 PS.RA @ m\n7109 !!DBSM/S!! ; PS.RA {row.signatur}\n"
    
    if pd.notna(row.komm_extern):
        pica += f"8034 {row.komm_extern}\n"

    pica += "\t\n"
    return pica

In [10]:
now = datetime.now()
for index, row in df.sample(2).iterrows():
    print(pica_schreiben(row))

0500 Pa
0501 unbewegtes Bild$bsti
0501 text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
1100 1789$n[zwischen 1789 und 1819]
1110 1789-1819 (Wirkungszeit des Papiermachers)*1789-1819$a1789$b1819$4vzta
1130 TB-papier-hr
1131 !041454197!
1132 a1-analog;a2-druck;f1-bild;f2-blatt;o-org
1500 /1zxx
1700 /1XA-DXDE
3000 !1051303966!$BPapiermacher$4ppm
3100 !1037995147!$BGeistiger Schöpfer$4cre
4000 Riesaufdruck des Papiermachers Johann Samuel Kutter, Motiv: gekrönter Reichsadler mit Zepter und Schwert, Brust belegt mit ISK-Monogramm / Obere Papiermühle im Ölschwang
4019 Druckgrafik$Bobja
4019 ISK$Bdevi
4046 Ravensburg : Obere Papiermühle im Ölschwang
4060 1 ungezähltes Blatt
4062 311 x 203 mm$b311$h203$4bltt
4062 143 x 141 mm$b143$h141$4bld
4105 !1257829130!
4201 Ein weiterer Riesaufdruck von der Papiermühle des Johann Samuel Kutter zeigt den gekrönten Reichsadler, womit auf den Status Ravensburgs als Reichsstadt hingewiesen wird. Adlerpapier dieser Papiermühle ist für 1790

In [41]:
now = datetime.now()
with open(f"../dat/riesaufdrucke-{now.strftime('%y-%m-%d-%H-%M-%S')}.dat","w") as f:
    with open(f"../dat/riesaufdrucke-recent.dat", 'w') as f2:
        for index, row in df.iterrows():
            f.write(pica_schreiben(row))
            f2.write(pica_schreiben(row))

with open(f"../dat/riesaufdrucke-recent-sample.dat", 'w') as f:
    for index, row in df.sample(5).iterrows():
        f.write(pica_schreiben(row))